In [1]:
import torch
import numpy as np

In [2]:
import pandas
#dataset taken from https://www.kaggle.com/yashsawarn/wifi-stretgth-for-rooms

USE_GINI = True

def read_dataset(csv_name = 'wifi_localization.txt'):
    """
    Reads a csv dataset
    returns it as a pytorch tensor
    """
    data_frame = pandas.read_table(csv_name, delim_whitespace=True, names=('A', 'B', 'C', 'D','E', 'F', 'G', 'ROOM'),
                       dtype={'A': np.int64, 'B': np.float64, 'C': np.float64, 'D': np.float64,'E': np.float64,'F': np.float64,'G': np.float64,'ROOM': np.float64})

    targets_torch = torch.tensor(data_frame['ROOM'].values)
    dataset_torch = torch.tensor(data_frame.values)

    return dataset_torch

# Implementación de la clasificación multi-clase con árboles de decisión

In [3]:
class Node_CART:
    def __init__(self, num_classes = 4, ref_CART = None, current_depth = 0):
        """
        Create the node attributes
        param num_classes: K number of classes to classify
        param ref_cart: reference to the tree containing the node
        param current_depth: current depth of the node in the tree
        """
        self.ref_CART = ref_CART
        self.threshold_value = 0
        self.feature_num = 0
        self.node_right = None
        self.node_left = None
        self.data_torch_partition = None
        self.gini = 0
        self.dominant_class = None
        self.accuracy_dominant_class = None
        self.num_classes = num_classes
        self.current_depth = current_depth

    def to_xml(self, current_str = ""):
        """
        Recursive function to write the node content to an xml formatted string
        param current_str : the xml content so far in the whole tree
        return the string with the node content
        """
        str_node = "<node><thresh>" + str(self.threshold_value) + "</thresh>" + "<feature>" + str(self.feature_num) + "</feature><depth>" + str(self.current_depth)+ "</depth>"
        str_node += "<gini>" + str(self.gini) + "</gini>"
        if(self.node_right != None):
            str_left = self.node_right.to_xml(current_str)
            str_node += str_left
        if(self.node_left != None):
            str_right = self.node_left.to_xml(current_str)
            str_node += str_right

        if(self.is_leaf()):
            str_node += "<dominant_class>" + str(self.dominant_class) + "</dominant_class><acc_dominant_class>"  + str(self.accuracy_dominant_class) + "</acc_dominant_class>"
        str_node += "</node>"
        return str_node

    def is_leaf(self):
        """
        Checks whether the node is a leaf
        """
        return (self.node_left == None and self.node_right == None)

    def create_with_children(self, data_torch, current_depth, list_selected_features = [], min_gini = 0.000001):
        """
        Creates a node by selecting the best feature and threshold, and if needed, creating its children
        param data_torch: dataset with the current partition to deal with in the node
        param current_depth: depth counter for the node
        param list_selected_features: list of selected features so far for the CART building process
        param min_gini: hyperparmeter selected by the user defining the minimum tolerated gini coefficient for a  node
        return the list of selected features so far
        """
        self.data_torch_partition = data_torch
        self.threshold_value, self.feature_num, self.gini = self.select_best_feature_and_thresh(data_torch, list_selected_features)
        # if not at max depth
        self.gini = self.calculate_gini(data_torch)
        if current_depth < self.ref_CART.max_CART_depth and self.gini >= min_gini:
            # if we found an appropiate feature + threshold
            if self.feature_num != float('inf') and self.threshold_value != float('inf'):
                left_data_torch = data_torch[data_torch[:,self.feature_num] <= self.threshold_value]
                right_data_torch = data_torch[data_torch[:,self.feature_num] > self.threshold_value]
                min_obs_per_leaf = self.ref_CART.get_min_observations()
                if (left_data_torch.shape[0] >= min_obs_per_leaf and right_data_torch.shape[0] >= min_obs_per_leaf):
                    list_selected_features.append(self.feature_num)
                    # lefts
                    self.node_left = Node_CART(ref_CART=self.ref_CART, current_depth=current_depth+1)
                    list_selected_features_left = self.node_left.create_with_children(left_data_torch, (current_depth+1), list_selected_features, min_gini)
                    # right
                    self.node_right = Node_CART(ref_CART=self.ref_CART, current_depth=current_depth+1)
                    list_selected_features_right = self.node_right.create_with_children(right_data_torch, (current_depth+1), list_selected_features, min_gini)
                    # combinar listas
                    list_selected_features = list(set(list_selected_features_left + list_selected_features_right))
        # dominant class
        labels, counts = torch.unique(data_torch[:,-1], return_counts=True)
        dominant_index = torch.argmax(counts)
        dominant_class = labels[dominant_index].item()
        self.dominant_class = dominant_class
        return list_selected_features

    def calculate_gini(self, data_partition_torch: torch.tensor, num_classes: int = 4):
        """
        Calculates the gini coefficient for a given partition with the given number of classes
        param data_partition_torch: current dataset partition as a tensor
        param num_classes: K number of classes to discriminate from
        returns the calculated gini coefficient
        """
        ROOM_COLUMN_INDEX = 7
        class_counts = torch.bincount(data_partition_torch[:, -1], minlength=num_classes + 1)
        class_counts = class_counts[1:]
        class_probabilities = class_counts.float() / data_partition_torch.shape[0]
        gini_coef = 1.0 - torch.sum(class_probabilities ** 2)
        return gini_coef

    def calculate_entropy(self, data_partition_torch, num_classes = 4):
        """
        Calculates the entropy for a given partition with the given number of classes
        param data_partition_torch: current dataset partition as a tensor
        param num_classes: K number of classes to discriminate from
        returns the calculated entropy
        """
        ROOM_COLUMN_INDEX = 7
        class_counts = torch.bincount(data_partition_torch[:, -1], minlength=num_classes + 1)
        class_counts = class_counts[1:]
        p_k = class_counts.float() / data_partition_torch.shape[0]
        p_k[p_k == 0.0] = 1.0 # Remove 0 value, since log2(0) = -inf and log2(1) = 0
        return -1*torch.sum(p_k*torch.log2(p_k))

    def evaluate_node(self, input_torch):
        """
        Evaluates an input observation within the node.
        If is not a leaf node, send it to the corresponding node
        return predicted label
        """
        feature_val_input = input_torch[self.feature_num]
        if(self.is_leaf()):
            return self.dominant_class
        else:
            if(feature_val_input <= self.threshold_value):
                return self.node_left.evaluate_node(input_torch)
            else:
                return self.node_right.evaluate_node(input_torch)

    def evaluate_optimal_loss(self, right_data_partition: torch.tensor,
                              left_data_partition: torch.tensor,
                              n_data_partition: int,
                              num_classes: int,
                              error_func):
        
        return (right_data_partition.shape[0]/n_data_partition) * error_func(right_data_partition, num_classes).item() + \
                (left_data_partition.shape[0]/n_data_partition) * error_func(left_data_partition, num_classes).item()

    def select_best_feature_and_thresh(self, data_torch, list_features_selected = [], num_classes = 4):
        """
        Selects the best feature and threshold that minimizes the gini coefficient
        param data_torch: dataset partition to analyze
        param list_features_selected list of features selected so far, thus must be ignored
        param num_classes: number of K classes to discriminate from
        return min_thresh, min_feature, min_gini found for the dataset partition when
        selecting the found feature and threshold
        """

        min_thresh = float('inf')
        min_feature = 0
        min_gini = float('inf')

        total_data_torch = data_torch.shape[1]

        # -1 to ignore label
        for feature in range(total_data_torch - 1):
          if (feature not in list_features_selected):
            for threshold in data_torch[:, feature]:
                left_data = data_torch[data_torch[:,feature] <= threshold]
                right_data = data_torch[data_torch[:,feature] > threshold]
                # solution to avoid division by 0 in calculate_gini in case a partition has 0 elements
                if (left_data.shape[0] > 0 and right_data.shape[0] > 0):
                    if USE_GINI:
                        weighted_err = self.evaluate_optimal_loss(right_data, left_data, total_data_torch, num_classes, self.calculate_gini)
                    else:
                        weighted_err = self.evaluate_optimal_loss(right_data, left_data, total_data_torch, num_classes, self.calculate_entropy)
                    if (weighted_err < min_gini):
                        min_thresh = threshold
                        min_feature = feature
                        min_gini = weighted_err
        return (min_thresh, min_feature, min_gini)


class CART:
    def __init__(self, dataset_torch, max_CART_depth, min_observations = 2):
        """
        CART has only one root node
        """
        #min observations per node
        self.min_observations = min_observations
        self.list_selected_features = []
        self.root = Node_CART(num_classes = 4, ref_CART = self, current_depth = 0)
        self.max_CART_depth = max_CART_depth
        print(self.list_selected_features)
        

    def get_root(self):
        """
        Gets tree root
        """
        return self.root

    def get_min_observations(self):
        """
        return min observations per node
        """
        return self.min_observations

    def get_max_depth(self):
        """
        Gets the selected max depth of the tree
        """
        return self.max_CART_depth

    def build_CART(self, data_torch):
        """
        Build CART from root
        """
        self.list_selected_features = self.root.create_with_children(data_torch, current_depth = 0, list_selected_features=self.list_selected_features)

    def to_xml(self, xml_file_name):
        """
        write Xml file with tree content
        """
        str_nodes = self.root.to_xml()
        file = open(xml_file_name,"w+")
        file.write(str_nodes)
        file.close()
        return str_nodes

    def evaluate_input(self, input_torch):
        """
        Evaluate a specific input in the tree and get the predicted class
        """
        return self.root.evaluate_node(input_torch)

def train_CART(dataset_torch, name_xml = "", max_CART_depth = 3, min_obs_per_leaf = 2):
    """
    Train CART model
    """
    tree = CART(dataset_torch = dataset_torch, max_CART_depth = max_CART_depth, min_observations =  min_obs_per_leaf)
    tree.build_CART(dataset_torch)
    if(not name_xml == ""):
        tree.to_xml(name_xml)
    return tree

def test_CART(tree, testset_torch):
    """
    Test a previously built CART
    """
    #ROOM_COLUMN_INDEX = 7
    ROOM_COLUMN_INDEX = -1
    class_columns = testset_torch[:, -1].int()
    num_classes = torch.bincount(class_columns)[1:].shape[0]
    n = testset_torch.shape[0]
    predicted_values = []
    c = 0
    for current_observation in testset_torch:
        real_value = current_observation[ROOM_COLUMN_INDEX].item()
        predicted_value = tree.evaluate_input(current_observation)
        predicted_values.append(predicted_value)
        c = c+1 if predicted_value == real_value else c
        #print('predicted_value=[{}], num_classes=[{}], real_value=[{}]'.format(predicted_value, num_classes, real_value))
    return c/n, predicted_values

## Gini Unit Test

In [4]:
import unittest

class GiniUnitTest(unittest.TestCase):

    def test_singleClassOneData(self):
      data = torch.tensor([[-64, -56,	-61,	-66,	-71,	-82,	-81,	1]])
      node = Node_CART()
      gini_result = node.calculate_gini(data, num_classes=1)
      self.assertTrue(torch.equal(gini_result, torch.tensor(0.0)))

    def test_twoClassesOneDataPerClass(self):
      data = torch.tensor([[-64, -56,	-61,	-66,	-71,	-82,	-81,	1], [-64, -56,	-61,	-66,	-71,	-82,	-81,	2]])
      node = Node_CART()
      gini_result = node.calculate_gini(data, num_classes=2)
      self.assertTrue(torch.equal(gini_result, torch.tensor(0.5)))

    def test_twoClassesOnlyOneClassWithData(self):
      data = torch.tensor([[-64, -56,	-61,	-66,	-71,	-82,	-81,	1], [-64, -56,	-61,	-66,	-71,	-82,	-81,	1]])
      node = Node_CART()
      gini_result = node.calculate_gini(data, num_classes=2)
      self.assertTrue(torch.equal(gini_result, torch.tensor(0.0)))

    def test_fourClassesOneDataPerClass(self):
      data = torch.tensor([[-64, -56,	-61,	-66,	-71,	-82,	-81,	1], [-64, -56,	-61,	-66,	-71,	-82,	-81,	2], [-64, -56,	-61,	-66,	-71,	-82,	-81,	3], [-64, -56,	-61,	-66,	-71,	-82,	-81,	4]])
      node = Node_CART()
      gini_result = node.calculate_gini(data, num_classes=4)
      self.assertTrue(torch.equal(gini_result, torch.tensor(0.75)))

    def test_fourClassesOnlyTwoClassesWithData(self):
      data = torch.tensor([[-64, -56,	-61,	-66,	-71,	-82,	-81,	1], [-64, -56,	-61,	-66,	-71,	-82,	-81,	1], [-64, -56,	-61,	-66,	-71,	-82,	-81,	4], [-64, -56,	-61,	-66,	-71,	-82,	-81,	4]])
      node = Node_CART()
      gini_result = node.calculate_gini(data, num_classes=4)
      self.assertTrue(torch.equal(gini_result, torch.tensor(0.5)))

    def test_fourClassesOnlyThreeClassesWithData(self):
      data = torch.tensor([[-64, -56,	-61,	-66,	-71,	-82,	-81,	1], [-64, -56,	-61,	-66,	-71,	-82,	-81,	1], [-64, -56,	-61,	-66,	-71,	-82,	-81,	3], [-64, -56,	-61,	-66,	-71,	-82,	-81,	4]])
      node = Node_CART()
      gini_result = node.calculate_gini(data, num_classes=4)
      self.assertTrue(torch.equal(gini_result, torch.tensor(0.625)))

## Entropy Unit Test

In [5]:
class EntropyUnitTest(unittest.TestCase):
    def test_twoClasses5and9(self):
      data = torch.tensor([
        [-64, -56,	-61,	-66,	-71,	-82,	-81,	1],
        [-64, -56,	-61,	-66,	-71,	-82,	-81,	1],
        [-64, -56,	-61,	-66,	-71,	-82,	-81,	1],
        [-64, -56,	-61,	-66,	-71,	-82,	-81,	1],
        [-64, -56,	-61,	-66,	-71,	-82,	-81,	1],
        [-64, -56,	-61,	-66,	-71,	-82,	-81,	2],
        [-64, -56,	-61,	-66,	-71,	-82,	-81,	2],
        [-64, -56,	-61,	-66,	-71,	-82,	-81,	2],
        [-64, -56,	-61,	-66,	-71,	-82,	-81,	2],
        [-64, -56,	-61,	-66,	-71,	-82,	-81,	2],
        [-64, -56,	-61,	-66,	-71,	-82,	-81,	2],
        [-64, -56,	-61,	-66,	-71,	-82,	-81,	2],
        [-64, -56,	-61,	-66,	-71,	-82,	-81,	2],
        [-64, -56,	-61,	-66,	-71,	-82,	-81,	2]
      ])
      node = Node_CART()
      entropy_result = node.calculate_entropy(data, num_classes=2)
      self.assertAlmostEqual(entropy_result.item(), 0.94, delta=0.01)

    def test_twoClassesEqualItems(self):
      data = torch.tensor([
        [-64, -56,	-61,	-66,	-71,	-82,	-81,	1],
        [-64, -56,	-61,	-66,	-71,	-82,	-81,	1],
        [-64, -56,	-61,	-66,	-71,	-82,	-81,	2],
        [-64, -56,	-61,	-66,	-71,	-82,	-81,	2]
      ])
      node = Node_CART()
      entropy_result = node.calculate_entropy(data, num_classes=2)
      self.assertAlmostEqual(entropy_result.item(), 1.0, delta=0.01)

    def test_twoClassesAllItemsOneClass(self):
      data = torch.tensor([
        [-64, -56,	-61,	-66,	-71,	-82,	-81,	1],
        [-64, -56,	-61,	-66,	-71,	-82,	-81,	1],
        [-64, -56,	-61,	-66,	-71,	-82,	-81,	1],
        [-64, -56,	-61,	-66,	-71,	-82,	-81,	1]
      ])
      node = Node_CART()
      entropy_result = node.calculate_entropy(data, num_classes=2)
      self.assertAlmostEqual(entropy_result.item(), 0.0, delta=0.01)

## Feat and Thresh Selection Unit test

In [6]:
class BestFeatAndThreshUnitTest(unittest.TestCase):

    def test_optimal_gini(self):
      right_data = torch.tensor([[-64, -56,	-61,	-66,	-71,	-82,	-81,	1], [-64, -56,	-61,	-66,	-71,	-82,	-81,	2], [-64, -56,	-61,	-66,	-71,	-82,	-81,	3], [-64, -56,	-61,	-66,	-71,	-82,	-81,	4]])
      left_data = torch.tensor([[-64, -56,	-61,	-66,	-71,	-82,	-81,	1], [-64, -56,	-61,	-66,	-71,	-82,	-81,	1], [-64, -56,	-61,	-66,	-71,	-82,	-81,	3], [-64, -56,	-61,	-66,	-71,	-82,	-81,	4]])
      node = Node_CART()
      result = node.evaluate_optimal_loss(right_data, left_data, n_data_partition=8, num_classes=4, error_func=node.calculate_gini)
      self.assertEqual(result, 0.6875)

    def test_twoClassesTwoElementsPerClassGini(self):
      global USE_GINI
      USE_GINI = True
      data = torch.tensor([ [-64, -56,	-61,	-66,	-71,	-82,	-81,  1],
                            [-50, -14,	-1,	-90,	-100,	-80,	-84,	2],
                            [-25, -50,	-100,	-9,	-10,	-85,	-70,	2],
                            [-64, -56,	-61,	-66,	-71,	-82,	-81,  1]])
      node = Node_CART()
      ((min_thresh, min_feature, min_gini)) = node.select_best_feature_and_thresh(data, num_classes=2)
      self.assertEqual(min_thresh, torch.tensor(-64))
      self.assertEqual(min_feature, torch.tensor(0.0))
      self.assertEqual(min_gini, torch.tensor(0.0))

    def test_twoClassesTwoElementsPerClassEntropy(self):
      global USE_GINI
      USE_GINI = False
      data = torch.tensor([ [1, 1, 1,	1, 1,	1, 1,	1],
                            [1, 1, 0,	1, 1,	1, 1,	1],
                            [1, 1, 1,	1, 1,	1, 1,	2],
                            [1, 1, 1,	1, 1,	1, 1,	2]])
      node = Node_CART()
      ((min_thresh, min_feature, min_gini)) = node.select_best_feature_and_thresh(data, num_classes=2)
      self.assertEqual(min_thresh, torch.tensor(0.))
      self.assertEqual(min_feature, torch.tensor(2))
      self.assertEqual(min_gini, torch.tensor(0.34436094760894775))

## Create with Children Unit test

In [7]:
class CreateWithChildrenUnitTest(unittest.TestCase):
    def test_twoClassesTwoDataPerClass(self):
      data = torch.tensor([ [-64, -56,	-61,	-90,	-71,	-82,	-81,  1],
                            [-64, -14,	-61,	-90,	-71,	-82,	-81,  2],
                            [-64, -56,	-61,	-90,	-71,	-82,	-81,  2],
                            [-64, -56,	-60,	-90,	-71,	-82,	-81,  1]])
      cart = CART(data, 2, 1)
      cart.build_CART(data)
      self.assertEqual(cart.list_selected_features, [1, 2])

    def test_twoClassesTwoDataPerClass_depth(self):
      data = torch.tensor([ [-64, -56,	-61,	-90,	-71,	-82,	-81,  1],
                            [-64, -14,	-61,	-90,	-71,	-82,	-81,  2],
                            [-64, -56,	-61,	-90,	-71,	-82,	-81,  2],
                            [-64, -56,	-60,	-90,	-71,	-82,	-81,  1]])
      cart = CART(data, 1, 1)
      cart.build_CART(data)
      self.assertEqual(cart.list_selected_features, [1])


## test_CART Unit Test

In [8]:
class TestCARTUnitTest(unittest.TestCase):

    def test_cartWithTwoClassesRightPrediction(self):
      train_data = torch.tensor([[0, 1, 0,	1, 1,	1, 1,	1],
                                 [0, 0, 1,	1, 1,	1, 1,	2]])

      test_tree = train_CART(train_data, min_obs_per_leaf=1)
      test_data = torch.tensor([[1, 1, 0,	1, 1,	0, 1,	1],
                                [1, 0, 1,	1, 1,	1, 0,	2]])
      acc, predicted_values = test_CART(test_tree, test_data)
      self.assertEqual(acc, 1.0)
      self.assertEqual(predicted_values, [1, 2])


    def test_cartWithTwoClassesWrongTestLabel(self):
      train_data = torch.tensor([[0, 1, 0,	1, 1,	1, 1,	1],
                                 [0, 0, 1,	1, 1,	1, 1,	2]])

      test_tree = train_CART(train_data, min_obs_per_leaf=1)
      test_data = torch.tensor([[1, 1, 0,	1, 1,	0, 1,	1],
                                [1, 0, 1,	1, 1,	1, 0,	1]])
      acc, predicted_values = test_CART(test_tree, test_data)
      self.assertEqual(acc, 0.5)
      self.assertEqual(predicted_values, [1, 2])


    def test_cartWithFourClassesRightPrediction(self):
      train_data = torch.tensor([[0, 1, 1, 1, 1, 1, 1, 1],
                                 [1, 0, 1, 1, 1, 1, 1, 2],
                                 [1, 1, 0, 1, 1, 1, 1, 3],
                                 [1, 1, 1, 0, 1, 1, 1, 4]])
      test_tree = train_CART(train_data, max_CART_depth=5, min_obs_per_leaf=1)
      test_data = torch.tensor([[1, 0, 1,	1, 1,	1, 0,	2],
                                [1, 1, 1,	0, 1,	0, 1,	4]])
      test_tree.to_xml("michael.xml")
      acc, predicted_values = test_CART(test_tree, test_data)
      self.assertEqual(acc, 1.0)
      self.assertEqual(predicted_values, [2, 4])



## Ejecución Unit Tests

In [9]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_optimal_gini (__main__.BestFeatAndThreshUnitTest.test_optimal_gini) ... ok
test_twoClassesTwoElementsPerClassEntropy (__main__.BestFeatAndThreshUnitTest.test_twoClassesTwoElementsPerClassEntropy) ... ok
test_twoClassesTwoElementsPerClassGini (__main__.BestFeatAndThreshUnitTest.test_twoClassesTwoElementsPerClassGini) ... ok
test_twoClassesTwoDataPerClass (__main__.CreateWithChildrenUnitTest.test_twoClassesTwoDataPerClass) ... ok
test_twoClassesTwoDataPerClass_depth (__main__.CreateWithChildrenUnitTest.test_twoClassesTwoDataPerClass_depth) ... ok
test_twoClasses5and9 (__main__.EntropyUnitTest.test_twoClasses5and9) ... ok
test_twoClassesAllItemsOneClass (__main__.EntropyUnitTest.test_twoClassesAllItemsOneClass) ... ok
test_twoClassesEqualItems (__main__.EntropyUnitTest.test_twoClassesEqualItems) ... ok
test_fourClassesOneDataPerClass (__main__.GiniUnitTest.test_fourClassesOneDataPerClass) ... ok
test_fourClassesOnlyThreeClassesWithData (__main__.GiniUnitTest.test_fourClassesOnlyThree

[]
[]
[]
[]
[]


# Evaluación del CART

## Utils

In [10]:
def f1_score(input, target, epsilon = 1e-7):
    input = torch.tensor(input)
    target = torch.tensor(target)
    
    class_f1_scores = []
    for class_label in torch.unique(target):
        # true positives, false positives, and false negatives for the current class
        tp = torch.sum((input == class_label) & (target == class_label)).float()
        fp = torch.sum((input == class_label) & (target != class_label)).float()
        fn = torch.sum((input != class_label) & (target == class_label)).float()

        # precision and recall for the current class
        precision = tp / (tp + fp + epsilon)
        recall = tp / (tp + fn + epsilon)

        # Compute F1 score for the current class
        class_f1 = 2 * (precision * recall) / (precision + recall + epsilon)
        class_weight = torch.sum(target == class_label).float()
        class_f1_scores.append(class_f1 * class_weight)

    weighted_f1 = torch.sum(torch.tensor(class_f1_scores)) / torch.numel(target)
    return weighted_f1

def train_test_split(X_data, train_size=0.7): 
    rand_indices = torch.randperm(X_data.shape[0]) 
 
    # get split sizes 
    train_split = int(train_size * rand_indices.shape[0]) 
    test_split = rand_indices.shape[0] - train_split 
 
    # split data 
    train_split = X_data[rand_indices[:train_split]] 
    test_split = X_data[rand_indices[test_split:]] 
 
    # Split to follow scikit learns format. 
    X_train = train_split[:,:-1] 
    y_train = train_split[:,-1] 
    X_test = test_split[:,:-1] 
    y_test = test_split[:,-1] 
    return X_train, X_test, y_train, y_test

## 1-

In [11]:
dataset_torch = read_dataset().int()
print('dataset_torch = [\n{}\n]'.format(dataset_torch))

############################### USING GINI ################################
USE_GINI = True

print('[GINI] - Test CART with max_CART_depth=[3] and min_obs_per_leaf=[2]')
tree_3 = train_CART(dataset_torch, name_xml = "CART_example.xml", max_CART_depth=3, min_obs_per_leaf=2)
acc, predicted_values = test_CART(tree_3, dataset_torch)
f1 = f1_score(dataset_torch[:,-1], predicted_values) 

print(f'[GINI] - Accuracy rate=[{acc}], f1_score=[{f1}]')

#########

print('[GINI] - Test CART with max_CART_depth=[4] and min_obs_per_leaf=[2]')
tree_4 = train_CART(dataset_torch, name_xml = "CART_example.xml", max_CART_depth=4, min_obs_per_leaf=2)
acc, predicted_values = test_CART(tree_4, dataset_torch)
f1 = f1_score(dataset_torch[:,-1], predicted_values) 

print(f'[GINI] - Accuracy rate=[{acc}], f1_score=[{f1}]')

############################### USING ENTROPY ################################
USE_GINI = False

print('[ENTROPY] - Test CART with max_CART_depth=[3] and min_obs_per_leaf=[2]')
tree_3 = train_CART(dataset_torch, name_xml = "CART_example.xml", max_CART_depth=3, min_obs_per_leaf=2)
acc, predicted_values = test_CART(tree_3, dataset_torch)
f1 = f1_score(dataset_torch[:,-1], predicted_values) 

print(f'[ENTROPY] - Accuracy rate=[{acc}], f1_score=[{f1}]')

#########

print('[ENTROPY] - Test CART with max_CART_depth=[4] and min_obs_per_leaf=[2]')
tree_4 = train_CART(dataset_torch, name_xml = "CART_example.xml", max_CART_depth=4, min_obs_per_leaf=2)
acc, predicted_values = test_CART(tree_4, dataset_torch)
f1 = f1_score(dataset_torch[:,-1], predicted_values) 

print(f'[ENTROPY] - Accuracy rate=[{acc}], f1_score=[{f1}]')

dataset_torch = [
tensor([[-64, -56, -61,  ..., -82, -81,   1],
        [-68, -57, -61,  ..., -85, -85,   1],
        [-63, -60, -60,  ..., -85, -84,   1],
        ...,
        [-62, -59, -46,  ..., -87, -88,   4],
        [-62, -58, -52,  ..., -90, -85,   4],
        [-59, -50, -45,  ..., -88, -87,   4]], dtype=torch.int32)
]
[GINI] - Test CART with max_CART_depth=[3] and min_obs_per_leaf=[2]
[]


C:\Users\pavel\AppData\Local\Temp\ipykernel_31488\3724860516.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input)


[GINI] - Accuracy rate=[0.9355], f1_score=[0.9356065392494202]
[GINI] - Test CART with max_CART_depth=[4] and min_obs_per_leaf=[2]
[]
[GINI] - Accuracy rate=[0.7515], f1_score=[0.8289557695388794]
[ENTROPY] - Test CART with max_CART_depth=[3] and min_obs_per_leaf=[2]
[]
[ENTROPY] - Accuracy rate=[0.9445], f1_score=[0.9445338249206543]
[ENTROPY] - Test CART with max_CART_depth=[4] and min_obs_per_leaf=[2]
[]
[ENTROPY] - Accuracy rate=[0.751], f1_score=[0.8284593224525452]


## -2

In [12]:
dataset_torch = read_dataset().int()
print('dataset_torch = [\n{}\n]'.format(dataset_torch))

############################### USING GINI ################################
USE_GINI = True

print('[GINI] - Test CART with max_CART_depth=[3] and min_obs_per_leaf=[2]')

X_train, X_test, y_train, y_test = train_test_split(dataset_torch, train_size=0.7)
dataset_train = torch.column_stack((X_train, y_train))
dataset_test = torch.column_stack((X_test, y_test)) 
tree_3 = train_CART(dataset_train, name_xml = "CART_example.xml", max_CART_depth=3, min_obs_per_leaf=2)
acc, predicted_values = test_CART(tree_3, dataset_test)
f1 = f1_score(dataset_test[:,-1], predicted_values) 

print(f'[GINI] - Accuracy rate=[{acc}], f1_score=[{f1}]')

#########

print('[GINI] - Test CART with max_CART_depth=[4] and min_obs_per_leaf=[2]')
X_train, X_test, y_train, y_test = train_test_split(dataset_torch, train_size=0.7)
dataset_train = torch.column_stack((X_train, y_train))
dataset_test = torch.column_stack((X_test, y_test)) 
tree_4 = train_CART(dataset_train, name_xml = "CART_example.xml", max_CART_depth=4, min_obs_per_leaf=2)
acc, predicted_values = test_CART(tree_4, dataset_test)
f1 = f1_score(dataset_test[:,-1], predicted_values) 

print(f'[GINI] - Accuracy rate=[{acc}], f1_score=[{f1}]')

############################### USING ENTROPY ################################
USE_GINI = False

print('[ENTROPY] - Test CART with max_CART_depth=[3] and min_obs_per_leaf=[2]')
X_train, X_test, y_train, y_test = train_test_split(dataset_torch, train_size=0.7)
dataset_train = torch.column_stack((X_train, y_train))
dataset_test = torch.column_stack((X_test, y_test)) 
tree_3 = train_CART(dataset_train, name_xml = "CART_example.xml", max_CART_depth=3, min_obs_per_leaf=2)
acc, predicted_values = test_CART(tree_3, dataset_test)
f1 = f1_score(dataset_test[:,-1], predicted_values) 

print(f'[ENTROPY] - Accuracy rate=[{acc}], f1_score=[{f1}]')

#########

print('[ENTROPY] - Test CART with max_CART_depth=[4] and min_obs_per_leaf=[2]')
X_train, X_test, y_train, y_test = train_test_split(dataset_torch, train_size=0.7)
dataset_train = torch.column_stack((X_train, y_train))
dataset_test = torch.column_stack((X_test, y_test)) 
tree_4 = train_CART(dataset_train, name_xml = "CART_example.xml", max_CART_depth=4, min_obs_per_leaf=2)
acc, predicted_values = test_CART(tree_4, dataset_test)
f1 = f1_score(dataset_test[:,-1], predicted_values) 

print(f'[ENTROPY] - Accuracy rate=[{acc}], f1_score=[{f1}]')

dataset_torch = [
tensor([[-64, -56, -61,  ..., -82, -81,   1],
        [-68, -57, -61,  ..., -85, -85,   1],
        [-63, -60, -60,  ..., -85, -84,   1],
        ...,
        [-62, -59, -46,  ..., -87, -88,   4],
        [-62, -58, -52,  ..., -90, -85,   4],
        [-59, -50, -45,  ..., -88, -87,   4]], dtype=torch.int32)
]
[GINI] - Test CART with max_CART_depth=[3] and min_obs_per_leaf=[2]
[]


C:\Users\pavel\AppData\Local\Temp\ipykernel_31488\3724860516.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input)


[GINI] - Accuracy rate=[0.91], f1_score=[0.9105929136276245]
[GINI] - Test CART with max_CART_depth=[4] and min_obs_per_leaf=[2]
[]
[GINI] - Accuracy rate=[0.75], f1_score=[0.8259315490722656]
[ENTROPY] - Test CART with max_CART_depth=[3] and min_obs_per_leaf=[2]
[]
[ENTROPY] - Accuracy rate=[0.93], f1_score=[0.9295657873153687]
[ENTROPY] - Test CART with max_CART_depth=[4] and min_obs_per_leaf=[2]
[]
[ENTROPY] - Accuracy rate=[0.7321428571428571], f1_score=[0.8139196634292603]


## Optuna

In [20]:
import optuna

def objective(trial):
    # hyperparams to optim
    max_depth = trial.suggest_int('max_depth', 2, 6)
    min_obs_per_node = trial.suggest_int('min_obs_per_node', 1, 100)
    
    # train cart
    X_train, X_test, y_train, y_test = train_test_split(dataset_torch, train_size=0.7)
    dataset_train = torch.column_stack((X_train, y_train))
    dataset_test = torch.column_stack((X_test, y_test)) 
    tree = train_CART(dataset_torch,
                        name_xml = "CART_example.xml",
                        max_CART_depth=max_depth,
                        min_obs_per_leaf=min_obs_per_node)
    acc, predicted_values = test_CART(tree_3, dataset_test)
    f1 = f1_score(dataset_test[:,-1], predicted_values)
    #return f1
    return acc

study = optuna.create_study(study_name='my_study2', storage='sqlite:///sqlitedb.db', load_if_exists=True, direction='maximize')
study.optimize(objective, n_trials=10)

print(study.best_params) 

[I 2024-04-07 18:32:11,808] A new study created in RDB with name: my_study2


[]


C:\Users\pavel\AppData\Local\Temp\ipykernel_31488\3724860516.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input)
[I 2024-04-07 18:32:26,546] Trial 0 finished with value: 0.9328571428571428 and parameters: {'max_depth': 3, 'min_obs_per_node': 81}. Best is trial 0 with value: 0.9328571428571428.


[]


[I 2024-04-07 18:32:40,419] Trial 1 finished with value: 0.9335714285714286 and parameters: {'max_depth': 2, 'min_obs_per_node': 24}. Best is trial 1 with value: 0.9335714285714286.


...

[]


[I 2024-04-07 18:34:17,061] Trial 8 finished with value: 0.9307142857142857 and parameters: {'max_depth': 6, 'min_obs_per_node': 33}. Best is trial 5 with value: 0.9357142857142857.


[]


[I 2024-04-07 18:34:32,608] Trial 9 finished with value: 0.9335714285714286 and parameters: {'max_depth': 4, 'min_obs_per_node': 95}. Best is trial 5 with value: 0.9357142857142857.


{'max_depth': 2, 'min_obs_per_node': 81}
